In [1]:
import rospy
import moveit_commander
from geometry_msgs.msg import Pose
from tf.transformations import euler_from_quaternion, quaternion_from_euler

## ノードの初期化

In [2]:
rospy.init_node("robot_command_init")

## ロボット情報の取得

In [3]:
robot = moveit_commander.RobotCommander()

[ WARN] [1675367601.964468443, 1982.493000000]: Kinematics solver doesn't support #attempts anymore, but only a timeout.
Please remove the parameter '/robot_description_kinematics/manipulator/kinematics_solver_attempts' from your configuration.


In [4]:
robot.get_group_names()

['manipulator', 'tool']

## 制御したいリンクの設定

In [5]:
manipulator = moveit_commander.MoveGroupCommander('manipulator')
tool = moveit_commander.MoveGroupCommander('tool')

## 現在の関節情報の取得

In [6]:
manipulator.get_current_joint_values()

[2.2960055378186084,
 2.3330958660959276,
 2.072126889664773,
 -0.3920130305751073,
 -1.030044963443288,
 2.1695561732018978]

In [7]:
tool.get_current_joint_values()

[2.1695561732018995]

## 現在のXYZ座標と姿勢の取得

In [8]:
manipulator.get_current_pose()

header: 
  seq: 0
  stamp: 
    secs: 1983
    nsecs: 872000000
  frame_id: "world"
pose: 
  position: 
    x: 0.3000418414346732
    y: -0.3000738148722561
    z: 0.4999927557576372
  orientation: 
    x: 0.4203412845125747
    y: 0.4205352628005871
    z: -0.22967417696223735
    w: 0.7705277864634595

In [9]:
tool.get_current_pose()

header: 
  seq: 0
  stamp: 
    secs: 1983
    nsecs: 892000000
  frame_id: "world"
pose: 
  position: 
    x: 0.3000418414346727
    y: -0.30007381487225565
    z: 0.4999927557576374
  orientation: 
    x: 0.42034128451257396
    y: 0.4205352628005869
    z: -0.22967417696223758
    w: 0.77052778646346

## ホームに移動

In [10]:
Home = [0.0, 7.669904334761668e-06, -1.5707907676696777, 0.0, -1.5707603693008423, -6.447359919548035e-05]
manipulator.set_joint_value_target(Home)
manipulator.go()

True

## 座標移動

In [11]:
getPose = manipulator.get_current_pose().pose
getPose.position.x += (100 / 1000) # X方向に100mm 移動
getPose.position.y += (100 / 1000) # Y方向に100mm 移動
getPose.position.z += (100 / 1000) # Z方向に100mm 移動

In [12]:
manipulator.set_pose_target(getPose)
manipulator.go()

True

## 手先の位置を指定

In [13]:
manipulator.set_position_target([0.1, 0.5, 0.1])
manipulator.go()

True

## 手先の姿勢を指定

In [14]:
manipulator.set_rpy_target( [ 0.0, -2.36, 0.0 ] )
manipulator.go()

True

## 手先の位置と姿勢を指定

In [15]:
manipulator.set_pose_target( [ 0.1, -0.1, 0.1, 0, -1.57, 0 ] )
manipulator.go()

True

## 位置・姿勢を指定

In [16]:
manipulator.set_pose_target( [ 0.1, -0.1, 0.2, 0, -3.14, 0 ] )
manipulator.go()

True

##  位置とクォータニオン

In [17]:
manipulator.set_pose_target( [ 0.2, -0.3, 0.4, 0.0, -1.0, 0.0, 0.0] )
manipulator.go()

True

## 座標指定

In [18]:
pose = Pose()
pose.position.x = 0.3
pose.position.y = -0.3
pose.position.z = 0.5

quat = quaternion_from_euler(1, 1, 0)
print(quat)
pose.orientation.x = quat[0]
pose.orientation.y = quat[1]
pose.orientation.z = quat[2]
pose.orientation.w = quat[3]

manipulator.set_pose_target(pose)
manipulator.go()

[ 0.42073549  0.42073549 -0.22984885  0.77015115]


True

## 間隔移動

In [42]:
import copy

# Go Home
manipulator.set_joint_value_target(Home)
manipulator.go()

pose = manipulator.get_current_pose().pose
for y in range(0, 4):
    _pose = copy.deepcopy(pose)
    _pose.position.y += 20 / 1000 * y # 20mm 移動
    manipulator.set_joint_value_target(_pose)
    manipulator.go()
    for x in range(0, 4):
        _pose.position.x += 65 / 1000 * x # 65mm 移動
        manipulator.set_joint_value_target(_pose)
        manipulator.go()